### Build a decision tree from scratch

Here we build a decision tree with GINI index and python from scratch, for those interested on creating it in programming level.


Credit to [Ryan's blog](https://www.ryanxin.cn/archives/283)

In [16]:
from math import log2

class node:
    def __init__(self):
        # Class tag or feature
        self.target = None
        # Features to fetch
        self.condition = None
        # Left subtree, when taking feature values
        self.left = None
        # Right subtree, when no feature is taken
        self.right = None

    def generate(self, dataSet, features, e=0):
        
        x_ranges = [set([i[j] for i in dataSet]) for j in range(len(dataSet[0]) - 1)]   # The set of feature fetches
        y_ranges = set([i[-1] for i in dataSet])  # The set of class tag fetches

        # If all instances belong to the same class, use that class as the class tag
        t = y_ranges
        if len(t) == 1:
            self.target = t.pop()
            return
        
        # If the feature set is empty, then the class with the largest number of instances in the dataset is used as the class tag
        if not features:
            t_dict = {}
            for x in y_ranges:
                t_dict[x] = len([i[-1] for i in dataSet if i[-1] == x])
            t_ls = list(t_dict.items())
            t_ls.sort(key=lambda a: a[1])
            self.target = t_ls[-1][0]
            return

        # Calculate the GINI index for each feature taken
        gini = {}
        for f in range(len(features)):
            gini[features[f]] = {}
            for value in x_ranges[f]:
                part_y1 = len([x[f] for x in dataSet if x[f] == value]) / len(dataSet)
                part_y2 = 1 - sum([(len([x[-1] for x in dataSet if x[f] == value and x[-1] == a]) / len([x[-1] for x in dataSet if x[f] == value])) ** 2 for a in y_ranges])
                part_n1 = len([x[f] for x in dataSet if x[f] != value]) / len(dataSet)
                part_n2 = 1 - sum([(len([x[-1] for x in dataSet if x[f] != value and x[-1] == a]) / len([x[-1] for x in dataSet if x[f] != value])) ** 2 for a in y_ranges])
                gini[features[f]][value] = part_y1 * part_y2 + part_n1 * part_n2
        
        # Sort to find the feature with the smallest GINI index and take the value
        gini_list = []
        for item in gini.items():
            for i in item[1].items():
                gini_list.append((item[0],)+i)
        gini_list.sort(key=lambda a: a[-1])

        # If the GINI index is less than the threshold, then the class with the largest number of instances in the dataset is used as the class tag
        if gini_list[0][-1] < e:
            t_dict = {}
            for x in y_ranges:
                t_dict[x] = len([i[-1] for i in dataSet if i[-1] == x])
            t_ls = list(t_dict.items())
            t_ls.sort(key=lambda a: a[1])
            self.target = t_ls[-1][0]
            return

        # Recursively construct the left and right child nodes
        self.target = gini_list[0][0]
        self.condition = gini_list[0][1]
        f_index = features.index(self.target)
        f = features[:]
        f.remove(self.target)
        d = [x[:] for x in dataSet if x[f_index] == self.condition]
        for j in range(len(d)):
            d[j].pop(f_index)
        self.left = node()
        self.left.generate(d,f,e)
        d = [x[:] for x in dataSet if x[f_index] != self.condition]
        for j in range(len(d)):
            d[j].pop(f_index)
        self.right = node()
        self.right.generate(d,f,e)


    def query(self, term):
        # Determine if the leaf node is queried
        if not self.condition:
            result = self.target
            return result
        else:
            if term[self.target] == self.condition:
                result = self.left.query(term)
                return result
            else:
                result = self.right.query(term)
                return result


if __name__ == "__main__":
    features = ['age', 'has a job', 'own a house', 'credit situation'] # feature_set
    train_data = [['Youth', 'no', 'no', 'average', 'no'], # training data set
                ['Youth', 'no', 'good', 'average', 'no'],
                ['Youth', 'yes', 'no', 'good', 'yes'],
                ['Youth', 'yes', 'yes', 'average', 'yes'],
                ['Youth', 'no', 'no','average', 'no'],
                ['Middle-aged','no', 'no', 'average','no'],
                ['Middle-aged','no','no','good','no'],
                ['Middle-aged','yes','yes','good','yes'],
                ['Middle-aged', 'no', 'yes', 'very good', 'yes'],
                ['Middle-aged', 'no', 'yes', 'very good', 'yes'],
                ['Elderly','no','yes','very good','yes'],
                ['Elderly', 'no', 'yes', 'good', 'yes'],
                ['Elderly', 'yes', 'no', 'good', 'yes'],
                ['Elderly','yes','no','very good','Yes'],
                ['Elderly', 'no', 'no', 'average', 'no']]

    n = node()
    n.generate(train_data,features,e=0)

    z = {'own a house': 'yes', 'has a job': 'no', 'age': 'Youth', 'credit situation': 'average'}
    print("Should the loan be given: "+n.query(z))

Should the loan be given: yes
